In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [2]:
pip install pyspark

     |████████████████████████████████| 215.7MB 59kB/s 
     |████████████████████████████████| 204kB 49.7MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=76b19e021483e7d7456818c78951e06783b5be367b3636622afd8206e34a1b5c
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
import numpy as np
import pandas as pd
import re
import sys
from pyspark import SparkConf, SparkContext

In [0]:
conf = SparkConf()
sc = SparkContext(conf = conf)

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
dataset_path= "/content/gdrive/My Drive/assign3/graph-small.txt"
iterations= 40
beta=0.8

In [0]:
def page_rank(dataset_path, iterations, beta):
  small1 =np.loadtxt(dataset_path)
  small=sc.parallelize(small1)
  l1=small.groupByKey().map(lambda x : (x[0],list(set(list(x[1])))))
  max_val=l1.count()
  l2=l1.map(lambda x: (x[0],len(x[1])))
  l3=l2.sortBy(lambda x: x[0])
  in1=small.map(lambda x : (x[0],x[1]))
  rddjoin=l3.join(in1)
  p1=rddjoin.map(lambda x: ((x[1][1],(x[0],x[1][0]))))
  p2=p1.groupByKey().map(lambda x : (x[0],list(set(list(x[1])))))
  p3=p2.sortBy(lambda x: x[0])
  p4=p3.map(lambda x: (x[0],[i[0] for i in x[1]],[1/i[1] for i in x[1]]))
  p5=p4.map(lambda x: (x[0],[[a,b] for a,b in zip(x[1],x[2])]))
  p6=p5.map(lambda x: (x[0],dict(x[1])))
  m_mat=p6.map(lambda y: np.array([y[1][i] if i in y[1].keys() else 0 for i in range(1,max_val+1)]))
  r=np.full((max_val,),1/max_val)
  for i in range(40):
    p=m_mat.map(lambda y: ((1-beta)/max_val)+(beta*np.dot(np.transpose(y),r)))
    r=np.array(p.collect())
    index=np.argsort(r)
    top5=list(index[-5:])
    print("iteration",i+1)
    j=4
    if(i==39):
      print("top 5 nodes after 40 iterations")
      while(j>=0):
        print(top5[j]+1, ":", r[top5[j]])
        j=j-1
      print("bottom 5 nodes after 40 iterations")
      for p in range(5):
        print(index[p] + 1,":",r[index[p]])
      break
    while(j>=0):
      print(top5[j]+1, ":", r[top5[j]])
      j=j-1

In [21]:
page_rank(dataset_path, iterations,beta)

iteration 1
263 : 0.0018759018759018757
502 : 0.001866349206349206
126 : 0.001843125763125763
285 : 0.001819076479076479
146 : 0.0018094549894549896
iteration 2
263 : 0.0020887152258490605
965 : 0.0019612553136778245
537 : 0.0018784860606765365
243 : 0.0018250404103778127
285 : 0.0018120241867479964
iteration 3
263 : 0.0020238066893208664
537 : 0.001930221482836505
965 : 0.0019265979884266348
243 : 0.0018556711996323286
285 : 0.001841159061219935
iteration 4
263 : 0.002012460909647064
537 : 0.001942342285906805
965 : 0.0019272412154498738
243 : 0.001851426809420244
285 : 0.0018288144290373816
iteration 5
263 : 0.002021301319496238
537 : 0.0019452806944060199
965 : 0.0019258364033716434
243 : 0.0018530462087332255
285 : 0.001827632665451319
iteration 6
263 : 0.002020648241069512
537 : 0.0019432292662412314
965 : 0.001925038850797009
243 : 0.0018527946111016682
285 : 0.0018273513860454682
iteration 7
263 : 0.002020369601235597
537 : 0.0019433250535116563
965 : 0.001925367516613498
243 : 